<a href="https://colab.research.google.com/github/hkdan502/NLP/blob/main/ner_deploy_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 패키지 설치
pip 명령어로 의존성 있는 패키지를 설치합니다.



In [1]:
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 KB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 KB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=be6836340213ca90410974eaa92283924d34233a520e7df4a404fea36a6cf31f
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c1

# 구글 드라이브 연동하기
모델 체크포인트 등을 저장해 둘 구글 드라이브를 연결합니다. 자신의 구글 계정에 적용됩니다.

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.

In [3]:
from ratsnlp.nlpbook.ner import NERDeployArguments
args = NERDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-ner",
    max_seq_length=64,
)

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-ner/epoch=1-val_loss=0.20-v1.ckpt
downstream_model_labelmap_fpath: /gdrive/My Drive/nlpbook/checkpoint-ner/label_map.txt


In [5]:
#토크나이저 로드
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

# 모델 로딩
파인튜닝을 마친 모델과 토크나이저를 읽어 들입니다.

In [4]:
import torch
from transformers import BertConfig, BertForTokenClassification

#체크포인트
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)

#BERT 모델 초기화
model = BertForTokenClassification(pretrained_model_config)
#모델에 체크포인트 추가
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [11]:
fine_tuned_model_ckpt['state_dict'].keys()

odict_keys(['model.bert.embeddings.position_ids', 'model.bert.embeddings.word_embeddings.weight', 'model.bert.embeddings.position_embeddings.weight', 'model.bert.embeddings.token_type_embeddings.weight', 'model.bert.embeddings.LayerNorm.weight', 'model.bert.embeddings.LayerNorm.bias', 'model.bert.encoder.layer.0.attention.self.query.weight', 'model.bert.encoder.layer.0.attention.self.query.bias', 'model.bert.encoder.layer.0.attention.self.key.weight', 'model.bert.encoder.layer.0.attention.self.key.bias', 'model.bert.encoder.layer.0.attention.self.value.weight', 'model.bert.encoder.layer.0.attention.self.value.bias', 'model.bert.encoder.layer.0.attention.output.dense.weight', 'model.bert.encoder.layer.0.attention.output.dense.bias', 'model.bert.encoder.layer.0.attention.output.LayerNorm.weight', 'model.bert.encoder.layer.0.attention.output.LayerNorm.bias', 'model.bert.encoder.layer.0.intermediate.dense.weight', 'model.bert.encoder.layer.0.intermediate.dense.bias', 'model.bert.encoder.la

# 레이블 맵 작성

범주 인덱스를 범주명과 매칭하는 사전을 만듭니다.

In [6]:
labels = [label.strip() for label in open(args.downstream_model_labelmap_fpath, "r").readlines()]

#정수 인덱스를 레이블에 매핑하는 사전
id_to_label = {} 
for idx, label in enumerate(labels):
  if "PER" in label:
    label = "인명"
  elif "LOC" in label:
    label = "지명"
  elif "ORG" in label:
    label = "기관명"
  elif "DAT" in label:
    label = "날짜"
  elif "TIM" in label:
    label = "시간"
  elif "DUR" in label:
    label = "기간"
  elif "MNY" in label:
    label = "통화"
  elif "PNT" in label:
    label = "비율"
  elif "NOH" in label:
    label = "기타 수량표현"
  elif "POH" in label:
    label = "기타"
  else:
    label = label
  id_to_label[idx] = label

# 인퍼런스 함수 선언
인퍼런스 함수를 선언합니다.

In [7]:
def inference_fn(sentence):
    inputs = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )
    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
        probs = outputs.logits[0].softmax(dim=1)
        top_probs, preds = torch.topk(probs, dim=1, k=1)
        tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
        predicted_tags = [id_to_label[pred.item()] for pred in preds]
        result = []
        for token, predicted_tag, top_prob in zip(tokens, predicted_tags, top_probs):
            if token not in [tokenizer.pad_token, tokenizer.cls_token, tokenizer.sep_token]:
                token_result = {
                    "token": token,
                    "predicted_tag": predicted_tag,
                    "top_prob": str(round(top_prob[0].item(), 4)),
                }
                result.append(token_result)
    return {
        "sentence": sentence,
        "result": result,
    }

In [12]:
inference_fn("그가 공부하는 분야는 어떤 분야인가요?")

{'sentence': '그가 공부하는 분야는 어떤 분야인가요?',
 'result': [{'token': '그가', 'predicted_tag': 'O', 'top_prob': '0.9992'},
  {'token': '공부', 'predicted_tag': 'O', 'top_prob': '0.9992'},
  {'token': '##하는', 'predicted_tag': 'O', 'top_prob': '0.9993'},
  {'token': '분야', 'predicted_tag': 'O', 'top_prob': '0.9991'},
  {'token': '##는', 'predicted_tag': 'O', 'top_prob': '0.9993'},
  {'token': '어떤', 'predicted_tag': 'O', 'top_prob': '0.9982'},
  {'token': '분야', 'predicted_tag': 'O', 'top_prob': '0.995'},
  {'token': '##인가요', 'predicted_tag': 'O', 'top_prob': '0.9993'},
  {'token': '?', 'predicted_tag': 'O', 'top_prob': '0.9993'}]}

In [13]:
inference_fn("그 제품은 삼성 물건인가요?")

{'sentence': '그 제품은 삼성 물건인가요?',
 'result': [{'token': '그', 'predicted_tag': 'O', 'top_prob': '0.9997'},
  {'token': '제품', 'predicted_tag': 'O', 'top_prob': '0.9993'},
  {'token': '##은', 'predicted_tag': 'O', 'top_prob': '0.9974'},
  {'token': '삼성', 'predicted_tag': '기관명', 'top_prob': '0.9486'},
  {'token': '물건', 'predicted_tag': 'O', 'top_prob': '0.9744'},
  {'token': '##인가요', 'predicted_tag': 'O', 'top_prob': '0.9971'},
  {'token': '?', 'predicted_tag': 'O', 'top_prob': '0.9978'}]}

In [14]:
inference_fn("코랩 로컬에서 실행 중인 웹서비스를 안전하게 외부에서 접근 가능하도록 해주는 도구입니다. ngrok을 실행하려면 회원가입 후 로그인을 한 뒤 이곳에 접속해 인증 토큰(authtoken)을 확인해야 합니다.")

{'sentence': '코랩 로컬에서 실행 중인 웹서비스를 안전하게 외부에서 접근 가능하도록 해주는 도구입니다. ngrok을 실행하려면 회원가입 후 로그인을 한 뒤 이곳에 접속해 인증 토큰(authtoken)을 확인해야 합니다.',
 'result': [{'token': '코', 'predicted_tag': '기타', 'top_prob': '0.4349'},
  {'token': '##랩', 'predicted_tag': '기타', 'top_prob': '0.5057'},
  {'token': '로', 'predicted_tag': '기타', 'top_prob': '0.4254'},
  {'token': '##컬', 'predicted_tag': '기타', 'top_prob': '0.4618'},
  {'token': '##에서', 'predicted_tag': 'O', 'top_prob': '0.9885'},
  {'token': '실행', 'predicted_tag': 'O', 'top_prob': '0.9994'},
  {'token': '중', 'predicted_tag': 'O', 'top_prob': '0.9997'},
  {'token': '##인', 'predicted_tag': 'O', 'top_prob': '0.9996'},
  {'token': '웹', 'predicted_tag': 'O', 'top_prob': '0.984'},
  {'token': '##서비스', 'predicted_tag': 'O', 'top_prob': '0.8085'},
  {'token': '##를', 'predicted_tag': 'O', 'top_prob': '0.9946'},
  {'token': '안전하게', 'predicted_tag': 'O', 'top_prob': '0.9989'},
  {'token': '외부', 'predicted_tag': 'O', 'top_prob': '0.9993'},
  {'token': '##에서', 'predicted

# 웹서비스 만들기 준비

`ngrok`은 코랩 로컬에서 실행 중인 웹서비스를 안전하게 외부에서 접근 가능하도록 해주는 도구입니다. `ngrok`을 실행하려면 [회원가입](https://dashboard.ngrok.com/signup) 후 [로그인](https://dashboard.ngrok.com/login)을 한 뒤 [이곳](https://dashboard.ngrok.com/get-started/your-authtoken)에 접속해 인증 토큰(authtoken)을 확인해야 합니다. 예를 들어 확인된 `authtoken`이 `test111`이라면 다음과 같이 실행합니다.

```bash
!mkdir /root/.ngrok2 && echo "authtoken: test111" > /root/.ngrok2/ngrok.yml
```

In [8]:
!mkdir /root/.ngrok2 && echo "authtoken: {이곳에 확인된 인증 토큰을 입력하세요}" > /root/.ngrok2/ngrok.yml

# 웹서비스 개시
아래처럼 실행해 인퍼런스 함수를 웹서비스로 만듭니다.

In [9]:
from ratsnlp.nlpbook.ner import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.ner.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
Exception in thread Thread-13:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/urllib3/connection.py", line 158, in _new_conn
    conn = connection.create_connection(
  File "/usr/local/lib/python3.8/dist-packages/urllib3/util/connection.py", line 80, in create_connection
    raise err
  File "/usr/local/lib/python3.8/dist-packages/urllib3/util/connection.py", line 70, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/urllib3/connectionpool.py", line 597, in urlopen
    httplib_response = self._make_request(conn, method, url,
  File "/usr/local/lib/python3.8/dist-packages/urllib3/connectionpool.py", line 354, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File 